In [1]:
import pandas as pd
import pickle
from helper_functions import Helper
from helper_functions import CoinAPI
import requests

In [33]:
if __name__ == "__main__":
    # import api key
    API_KEY_FP = '../../admin/coinapi.txt'
    with open(API_KEY_FP) as f:
        API_KEY = f.readlines()[0].strip()

    # Set args
    CW_IN_FP = '../data/derived/cm_to_coinapi_cw.pkl'
    ASSET_IN_FP = '../data/derived/asset_universe_dict.pickle'
    BASE_URL   = 'https://rest.coinapi.io/v1/'
    BASE_HEADERS = {'X-CoinAPI-Key': API_KEY}
    LEGIT_US_EXCHANGES = ['BINANCEUS', 'BITSTAMP', 'COINBASE', 'CRYPTOCOM', 'FTXUS', 
        'GEMINI', 'KRAKEN', 'KUCOIN']
    TARGET_FREQ = '1HRS'
    TIME_START = '2016-07-01'
    TIME_END = '2023-01-02'
    PANEL_OUT_FP = '../data/raw/coinapi_panel_hourly.pkl'
    MACRO_OUT_FP = '../data/raw/coinapi_macro_hourly.pkl'

    # confirm api is working
    url = 'https://www.coinapi.io/api/subscriptions/usage/rest/history'
    print(requests.get(url, headers=BASE_HEADERS).json())    
    
    # Import asset universe and cw
    cw_df = pd.read_pickle(CW_IN_FP)
    with open(ASSET_IN_FP, "rb") as f:
        asset_universe_dict = pickle.load(f)
    asset_universe_cm = Helper.findUniqueAssets(asset_universe_dict)
    asset_universe = list(cw_df[cw_df.asset_cm.isin(asset_universe_cm)].asset_coinapi.values)

    # pull relevant markets
    markets_df = CoinAPI.pullMarketInfo(BASE_URL, BASE_HEADERS, LEGIT_US_EXCHANGES, asset_universe)

    # pull coinapi aggregated prices
    prices_df = CoinAPI.pullExchangeRates(BASE_URL, BASE_HEADERS, ['USDC', 'USDT'] + asset_universe,
        TARGET_FREQ, TIME_START, TIME_END)
    macro_df, prices_df = CoinAPI.seperateMacroAndAssetRefPrices(prices_df)
    macro_df.to_pickle(MACRO_OUT_FP)

    # pull coinapi ohlcv
    raw_panel_df = CoinAPI.pullMarketData(BASE_URL, BASE_HEADERS,  list(markets_df.symbol_id.values), 
        TARGET_FREQ, TIME_START, TIME_END)

    # clean the panel
    panel_df = CoinAPI.cleanPanel(raw_panel_df, prices_df, macro_df, TARGET_FREQ)
    panel_df.to_pickle(PANEL_OUT_FP)